# **Imports**



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Conv2D,Add
from tensorflow.keras.layers import SeparableConv2D,ReLU
from tensorflow.keras.layers import BatchNormalization,MaxPool2D
from tensorflow.keras.layers import GlobalAvgPool2D
from tensorflow.keras import Model
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import LayerNormalization, Dense, Embedding, Layer, Dropout, Input
from tensorflow.keras.layers import MultiHeadAttention, Add, Flatten, Activation, MaxPooling2D, add, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, AdamW
# import tensorflow_addons as tfa
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_custom_objects
from PIL import Image
import numpy as np
from tensorflow.keras import backend, layers, models
import tensorflow as tf
import numpy as np
from PIL import Image
import os

In [ ]:
from skimage import io

# **Data Preprocessing**

In [ ]:
train = ImageDataGenerator(rescale=1/255.0)
trainset = train.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/ForgeryDetection/fold0',
    target_size=(160,160),
)

Found 3998 images belonging to 2 classes.


In [ ]:
batch = next(trainset)

In [ ]:
batch

# Xception Network

In [ ]:
TF_WEIGHTS_PATH_NO_TOP = '/content/drive/MyDrive/Colab Notebooks/ForgeryDetection/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'


def Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, **kwargs):
    """Instantiates the Xception architecture.

    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.

    Note that the default input image size for this model is 299x299.

    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(299, 299, 3)`.
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 71.
            E.g. `(150, 150, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True,
            and if no `weights` argument is specified.

    # Returns
        A Keras model instance.

    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
    """
    # backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = (256, 256, 3)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    channel_axis = 1 if backend.image_data_format() == 'channels_first' else -1

    x = layers.Conv2D(32, (3, 3),
                      strides=(2, 2),
                      use_bias=False,
                      name='block1_conv1')(img_input)
    x = layers.BatchNormalization(axis=channel_axis, name='block1_conv1_bn')(x)
    x = layers.Activation('relu', name='block1_conv1_act')(x)
    x = layers.Conv2D(64, (3, 3), use_bias=False, name='block1_conv2')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block1_conv2_bn')(x)
    x = layers.Activation('relu', name='block1_conv2_act')(x)

    residual = layers.Conv2D(128, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)

    x = layers.SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv1')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block2_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block2_sepconv2_act')(x)
    x = layers.SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv2')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block2_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block2_pool')(x)
    x = layers.add([x, residual])

    residual = layers.Conv2D(256, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)

    x = layers.Activation('relu', name='block3_sepconv1_act')(x)
    x = layers.SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv1')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block3_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block3_sepconv2_act')(x)
    x = layers.SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv2')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block3_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block3_pool')(x)
    x = layers.add([x, residual])

    residual = layers.Conv2D(728, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)

    x = layers.Activation('relu', name='block4_sepconv1_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv1')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block4_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block4_sepconv2_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv2')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block4_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block4_pool')(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv1')(x)
        x = layers.BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv1_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv2')(x)
        x = layers.BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv2_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv3')(x)
        x = layers.BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv3_bn')(x)

        x = layers.add([x, residual])

    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)

    x = layers.Activation('relu', name='block13_sepconv1_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv1')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block13_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block13_sepconv2_act')(x)
    x = layers.SeparableConv2D(1024, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv2')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block13_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block13_pool')(x)
    x = layers.add([x, residual])

    x = layers.SeparableConv2D(1536, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv1')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block14_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block14_sepconv1_act')(x)

    x = layers.SeparableConv2D(2048, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv2')(x)
    x = layers.BatchNormalization(axis=channel_axis, name='block14_sepconv2_bn')(x)
    x = layers.Activation('relu', name='block14_sepconv2_act')(x)

    # if include_top:
    #     x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    #     x = layers.Dense(classes, activation='softmax', name='predictions')(x)

    x = layers.GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    inputs = img_input
    # Create model.
    model = models.Model(inputs, x, name='xception')

    # Load weights.
    if weights == 'imagenet':
        weights_path = TF_WEIGHTS_PATH_NO_TOP
        model.load_weights(weights_path)
    elif weights is not None:
        model.load_weights(weights)

    # Set the layers to be non-trainable
    for layer in range(len(model.layers) - 1):
        if isinstance(model.layers[layer], (layers.BatchNormalization, layers.Conv2D, layers.SeparableConv2D)):
            model.layers[layer].trainable = False
    return model

# Alex Net

In [ ]:
def buildAlexNet(inputs):
    y = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
    y = BatchNormalization()(y)
    y = MaxPool2D(pool_size=(3,3), strides=(2,2))(y)
    y = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(y)
    y = BatchNormalization()(y)
    y = MaxPool2D(pool_size=(3,3), strides=(2,2))(y)
    y = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(y)
    y = BatchNormalization()(y)
    y = Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(y)
    y = BatchNormalization()(y)
    y = Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(y)
    y = BatchNormalization()(y)
    y = MaxPool2D(pool_size=(3,3), strides=(2,2))(y)
    y = Flatten()(y)
    y = Dense(4096, activation='relu')(y)
    y = Dropout(0.5)(y)
    y = Dense(4096, activation='relu')(y)
    y = Dropout(0.5)(y)
    return y


In [ ]:
def buildXception(inputs):

    channel_axis = -1
    x = Conv2D(32, (3, 3),
                      strides=(2, 2),
                      use_bias=False,
                      name='block1_conv1')(inputs)
    x = BatchNormalization(axis=channel_axis, name='block1_conv1_bn')(x)
    x = Activation('relu', name='block1_conv1_act')(x)
    x = Conv2D(64, (3, 3), use_bias=False, name='block1_conv2')(x)
    x = BatchNormalization(axis=channel_axis, name='block1_conv2_bn')(x)
    x = Activation('relu', name='block1_conv2_act')(x)

    residual = Conv2D(128, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False)(x)
    residual = BatchNormalization(axis=channel_axis)(residual)

    x = SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv1')(x)
    x = BatchNormalization(axis=channel_axis, name='block2_sepconv1_bn')(x)
    x = Activation('relu', name='block2_sepconv2_act')(x)
    x = SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv2')(x)
    x = BatchNormalization(axis=channel_axis, name='block2_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block2_pool')(x)
    x = add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False)(x)
    residual = BatchNormalization(axis=channel_axis)(residual)

    x = Activation('relu', name='block3_sepconv1_act')(x)
    x = SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv1')(x)
    x = BatchNormalization(axis=channel_axis, name='block3_sepconv1_bn')(x)
    x = Activation('relu', name='block3_sepconv2_act')(x)
    x = SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv2')(x)
    x = BatchNormalization(axis=channel_axis, name='block3_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block3_pool')(x)
    x = add([x, residual])

    residual = Conv2D(728, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False)(x)
    residual = BatchNormalization(axis=channel_axis)(residual)

    x = Activation('relu', name='block4_sepconv1_act')(x)
    x = SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv1')(x)
    x = BatchNormalization(axis=channel_axis, name='block4_sepconv1_bn')(x)
    x = Activation('relu', name='block4_sepconv2_act')(x)
    x = SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv2')(x)
    x = BatchNormalization(axis=channel_axis, name='block4_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block4_pool')(x)
    x = add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv1')(x)
        x = BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv1_bn')(x)
        x = Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv2')(x)
        x = BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv2_bn')(x)
        x = Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv3')(x)
        x = BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv3_bn')(x)

        x = add([x, residual])

    residual = Conv2D(1024, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False)(x)
    residual = BatchNormalization(axis=channel_axis)(residual)

    x = Activation('relu', name='block13_sepconv1_act')(x)
    x = SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv1')(x)
    x = BatchNormalization(axis=channel_axis, name='block13_sepconv1_bn')(x)
    x = Activation('relu', name='block13_sepconv2_act')(x)
    x = SeparableConv2D(1024, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv2')(x)
    x = BatchNormalization(axis=channel_axis, name='block13_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block13_pool')(x)
    x = add([x, residual])

    x = SeparableConv2D(1536, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv1')(x)
    x = BatchNormalization(axis=channel_axis, name='block14_sepconv1_bn')(x)
    x = Activation('relu', name='block14_sepconv1_act')(x)

    x = SeparableConv2D(2048, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv2')(x)
    x = BatchNormalization(axis=channel_axis, name='block14_sepconv2_bn')(x)
    x = Activation('relu', name='block14_sepconv2_act')(x)
    x = GlobalMaxPooling2D()(x)
    x = Flatten()(x)
    return x

# Hybridization

In [ ]:
with tf.device('/GPU:0'):
    xception = Xception()
    train = ImageDataGenerator(rescale=1/255.0)
    trainset = train.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/ForgeryDetection/fold',
        target_size=(256, 256),
        batch_size=10
    )
    features = xception.predict(trainset)

Found 40 images belonging to 2 classes.
4/4 [==============================] - 13s 2s/step


In [ ]:
features.shape

(3998, 2048)

In [ ]:
def buildXANet():
    # input layer

    inputs = Input(shape=(256, 256, 3))

    # SOTA Xception and AlexNet
    x = buildXception(inputs=inputs)
    y = buildAlexNet(inputs=inputs)

    # DNN
    z = layers.concatenate([x,y], axis=-1)
    z = Dense(1024, activation='relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(1024, activation='leaky_relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(2, activation='softmax')(z)

    # Model
    model = Model(inputs, z, name='XANet')

    return model

In [ ]:
batch_size=64

In [ ]:
with tf.device('/GPU:0'):
    # Model
    model = buildXANet()

    # Dataloader
    train = ImageDataGenerator(rescale=1/255.0)
    trainset = train.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/ForgeryDetection/fold',
        target_size=(256, 256),
        batch_size=32
    )

    # Training
    opt = Adam(
        learning_rate=0.0001,
    )
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(trainset, epochs=50)

Found 88 images belonging to 2 classes.
Epoch 1/50
3/3 [==============================] - 52s 4s/step - loss: 2.9591 - accuracy: 0.4432
Epoch 2/50
3/3 [==============================] - 2s 590ms/step - loss: 2.3239 - accuracy: 0.5909
Epoch 3/50
3/3 [==============================] - 2s 676ms/step - loss: 3.0854 - accuracy: 0.5341
Epoch 4/50
3/3 [==============================] - 2s 593ms/step - loss: 2.7320 - accuracy: 0.5341
Epoch 5/50
3/3 [==============================] - 2s 616ms/step - loss: 3.0258 - accuracy: 0.4432
Epoch 6/50
3/3 [==============================] - 2s 583ms/step - loss: 2.3731 - accuracy: 0.5795
Epoch 7/50
3/3 [==============================] - 2s 577ms/step - loss: 2.8757 - accuracy: 0.5682
Epoch 8/50
3/3 [==============================] - 2s 573ms/step - loss: 2.6163 - accuracy: 0.5568
Epoch 9/50
3/3 [==============================] - 2s 572ms/step - loss: 3.3311 - accuracy: 0.5682
Epoch 10/50
3/3 [==============================] - 2s 743ms/step - loss: 2.7410 

In [ ]:
model.summary()

# _**ViT**_




In [ ]:
num_classes = 2
input_shape = (256, 256, 3)

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 25
image_size = 256  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
class Patches(Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = Dense(units=projection_dim)
        self.position_embedding = Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def MLP(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = Dense(units, activation=tf.nn.gelu)(x)
        x = Dropout(dropout_rate)(x)
    return x

In [ ]:
def ModelViT():
    inputs = Input(shape=input_shape)

    # Create patches & Encode patches.
    patches = Patches(patch_size)(input)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):

        # Layer normalization 1.
        x1 = LayerNormalization(epsilon=1e-6)(encoded_patches)

        # Create a multi-head attention layer.
        attention_output = MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        x2 = Add()([attention_output, encoded_patches])

        # Layer normalization 2.
        x3 = LayerNormalization(epsilon=1e-6)(x2)

        # MLP.
        x3 = MLP(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2.
        encoded_patches = Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = Flatten()(representation)
    representation = Dropout(0.5)(representation)

    # MLP.
    features = MLP(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    logits = Dense(num_classes)(features)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)

    return model

In [ ]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=20)

with tf.device('/GPU:0'):
    # Model
    model = ModelViT()
    train = ImageDataGenerator(rescale=1/255.0)
    trainset = train.flow_from_directory(
          '/content/drive/MyDrive/Colab Notebooks/ForgeryDetection/fold',
          target_size=(256, 256),
          batch_size=32
      )
    # Training
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(trainset, steps_per_epoch=20, epochs=25)

ValueError: ignored

# _**XViT Network**_


In [ ]:
num_classes = 256
input_shape = (256, 256, 3)

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 25
image_size = 240  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
def Xception():
    input = Input(shape = input_shape)
    tensor, x = PreConv(input)
    x = ConvA(tensor, x)
    x = ConvB(x)
    output = ConvCOut(x)
    return  Model(inputs=input, outputs=output)

In [ ]:
def ModelXViT():
  Xception()

# NeNet

In [ ]:
batch_size=64
steps_per_epoch = 3998//batch_size

In [ ]:
def buildNeNet():
    # input layer

    inputs = Input(shape=(256, 256, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2d_64_3x3')(inputs)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='maxpool_1')(x)
    x = Conv2D(128, (5, 5), activation='relu', padding='same', name='conv2d_128_5x5')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='maxpool_2')(x)
    x = Conv2D(256, (11, 11), activation='relu', padding='same', name='conv2d_256_11x11')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='maxpool_3')(x)
    x = Conv2D(512, (21, 21), activation='relu', padding='same', name='conv2d_64_21x21')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='maxpool_4')(x)
    x = BatchNormalization(name='bn_1')(x)
    z = Flatten()(x)
    # DNN
    z = Dense(2048, activation='relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(2048, activation='relu')(z)
    z = Dropout(0.5)(z)
    z = Dense(1024, activation='relu')(z)
    z = Dense(2, activation='softmax')(z)

    # Model
    model = Model(inputs, z, name='NeNet')

    return model

In [ ]:
with tf.device('/GPU:0'):
    # Model
    model = buildNeNet()

    # Dataloader
    train = ImageDataGenerator(rescale=1/255.0)
    trainset = train.flow_from_directory(
        '/content/drive/MyDrive/Colab Notebooks/ForgeryDetection/fold',
        target_size=(256, 256),
        batch_size=32
    )

    # Training
    opt = Adam(
        learning_rate=0.0001,
    )
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(trainset, epochs=50)

Found 88 images belonging to 2 classes.
Epoch 1/50
3/3 [==============================] - 8s 1s/step - loss: 4.5115 - accuracy: 0.4659
Epoch 2/50
3/3 [==============================] - 5s 1s/step - loss: 1.5626 - accuracy: 0.5227
Epoch 3/50
3/3 [==============================] - 5s 1s/step - loss: 2.2851 - accuracy: 0.4773
Epoch 4/50
3/3 [==============================] - 5s 1s/step - loss: 1.9918 - accuracy: 0.5227
Epoch 5/50
3/3 [==============================] - 5s 1s/step - loss: 2.1328 - accuracy: 0.4886
Epoch 6/50
3/3 [==============================] - 5s 2s/step - loss: 2.0746 - accuracy: 0.4773
Epoch 7/50
3/3 [==============================] - 5s 1s/step - loss: 2.0411 - accuracy: 0.4545
Epoch 8/50
3/3 [==============================] - 5s 1s/step - loss: 1.5337 - accuracy: 0.4773
Epoch 9/50
3/3 [==============================] - 5s 1s/step - loss: 1.7441 - accuracy: 0.5568
Epoch 10/50
3/3 [==============================] - 5s 1s/step - loss: 1.4563 - accuracy: 0.5341
Epoch 11/

KeyboardInterrupt: ignored